In [2]:
import numpy as np
import pandas as pd


events = pd.read_csv('../data/events.csv')
events.head(2)

,user_id,item_id,rating,timestamp
0,0,1505,4,0
1,0,3669,3,1


In [3]:
item_features = pd.read_csv('../data/item_features.csv')
item_features.head(2)

,item_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [4]:
user_features = pd.read_csv('../data/user_features.csv')
user_features['gender'] = user_features['gender'].map({'F': 0, 'M': 1})
user_features.head(2)

,user_id,gender,age
0,4855,0,1
1,4065,1,56


In [5]:
events = pd.read_csv('../data/events.csv')
events = events.merge(item_features, on='item_id')

# домножаем на жанр на оценку
events_np = events[['genre_' + str(i) for i in range(18)]].values
rating_np = events['rating'].values
# если пользователь всем ставит низкие оценки, нужно это учесть
rating_norm = events.merge(events.groupby('user_id')['rating'].agg('mean').reset_index(), \
            how='left', on='user_id')['rating_y'].values

events[['genre_' + str(i) for i in range(18)]] = events_np * \
                              rating_np.reshape(-1, 1) / rating_norm.reshape(-1, 1)


events = events.groupby('user_id')[['genre_' + str(i) for i in range(18)]].agg('mean')

data = events.reset_index().merge(user_features).set_index('user_id')

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data = scaler.fit_transform(data)

In [8]:
data[:, -2]

array([0.62816964, 0.62816964, 0.62816964, ..., 0.62816964, 0.62816964,
       0.62816964])

In [10]:
import faiss
index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [11]:
distances, closest_neighbours = index.search(data, 30)
distances = distances[:, 1:]
closest_neighbours = closest_neighbours[:, 1:]

In [12]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm import tqdm

In [14]:
events = pd.read_csv('../data/events.csv')

seen_films = events.groupby('user_id').apply(lambda df: df['item_id'].values).values

def get_films_rec(neighbours_idxs, seen_films):
  neighbours = pd.DataFrame(data = neighbours_idxs, columns=['user_id'])
  films = neighbours.merge(events, how='left')
  films = films[films['rating'] > 3]
  films = films.item_id.value_counts()
  rec_hyp = films.index.values

  rec = []
  for i in rec_hyp:
    if not i in seen_films:
      rec.append(i)

    if len(rec) == 10:
      break

  return rec


answ = []
for i in tqdm(range(user_features.shape[0])):
  answ.append(get_films_rec(closest_neighbours[i], seen_films[i]))

100%|█████████████████████████████████████████████████████████████████████████████████| 6040/6040 [01:14<00:00, 81.58it/s]


In [15]:
sub = pd.read_csv('../data/submission_sample.csv')
sub.shape

(6040, 2)

In [16]:
sample_sub = pd.read_csv('../data/submission_sample.csv')
sample_sub.head(2)['item_id'].values

array(['0 1 2 3 4 5 6 7 8 9', '0 1 2 3 4 5 6 7 8 9'], dtype=object)

In [17]:
sample_sub['item_id'] = [' '.join(list(map(str, i))) for i in answ]

In [18]:
sample_sub.to_csv('submission_heuristic_final.csv', index=False)

In [19]:
sample_sub

,user_id,item_id
0,0,2732 2256 1811 1808 1543 331 790 69 2688 2630
1,1,463 1039 3529 208 3153 3046 124 1211 785 36
2,2,1831 640 2210 1211 1039 463 293 2646 2338 2256
3,3,3022 2603 3529 3046 2194 3327 183 2281 3005 3692
4,4,2732 1337 1583 3022 1030 3694 3350 1560 2297 2646
...,...,...
6035,6035,36 2510 2784 1337 3529 3234 584 2646 3013 1855
6036,6036,36 1039 2862 2564 3013 463 3529 2402 989 3153
6037,6037,1968 472 2402 2688 618 3269 3409 2054 3153 36
6038,6038,1583 1543 1615 640 476 1315 796 944 1337 3105
